In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import timedelta
from tqdm import tqdm

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)


# LOAD, Validate and basic data prep

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mashlyn/online-retail-ii-uci")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'online-retail-ii-uci' dataset.
Path to dataset files: /kaggle/input/online-retail-ii-uci


In [ ]:
import os
# List files in the downloaded path
os.listdir(path)

['online_retail_II.csv']

In [ ]:
import pandas as pd
import os

file_path = os.path.join(path, "online_retail_II.csv")
df = pd.read_csv(file_path)

In [ ]:
df = df.rename(columns={
    "Invoice": "InvoiceNo",
    "Customer ID": "CustomerID",
    "Price": "UnitPrice"
})

In [ ]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
import pandas as pd

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce")
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["UnitPrice"] = pd.to_numeric(df["UnitPrice"], errors="coerce")

In [ ]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [ ]:
REQUIRED_COLUMNS = [
    "InvoiceNo",
    "CustomerID",
    "InvoiceDate",
    "Quantity",
    "UnitPrice"
]

missing = set(REQUIRED_COLUMNS) - set(df.columns)

if missing:
    raise ValueError(f"❌ Missing columns: {missing}")
else:
    print("✅ Schema validation passed")

✅ Schema validation passed


In [ ]:
df[REQUIRED_COLUMNS].isnull().sum()

,0
InvoiceNo,0
CustomerID,243007
InvoiceDate,0
Quantity,0
UnitPrice,0


In [ ]:
INFERENCE_CUTOFF_DATE = pd.to_datetime("2011-12-01")

# Data Cleaning + Leakage Safe cut-off filtering

In [ ]:
""" Remove cancelled invoices
Cancelled orders do not represent revenue
"""


df = df[~df["InvoiceNo"].astype(str).str.startswith("C")]

In [ ]:
""" Remove Invalid Customers
Inference requires identified customers7n
"""

df = df.dropna(subset=["CustomerID"])
df["CustomerID"] = df["CustomerID"].astype(int)


In [ ]:
# Remove Invalid Transactions

df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

In [ ]:
# Recompute TotalSpend (Safety Step)

df["TotalSpend"] = df["Quantity"] * df["UnitPrice"]

In [ ]:
"""Apply Inference Cut-off Date (MOST IMPORTANT STEP)
This is where data leakage is prevented

From this point onward:
Only df_hist is used
Anything after cut-off is forbidden
"""

INFERENCE_CUTOFF_DATE = pd.to_datetime("2011-12-01")
df_hist = df[df["InvoiceDate"] < INFERENCE_CUTOFF_DATE]

In [ ]:
# Post-Cleaning Sanity Checks

print("Transactions after cleaning:", df_hist.shape[0])
print("Unique customers:", df_hist["CustomerID"].nunique())
print("Date range:", df_hist["InvoiceDate"].min(), "→", df_hist["InvoiceDate"].max())

Transactions after cleaning: 788244
Unique customers: 5850
Date range: 2009-12-01 07:45:00 → 2011-11-30 17:37:00


# CUSTOMER-LEVEL FEATURE ENGINEERING (INFERENCE)

At the end of this part, you will have a single row per customer, containing exactly the features your trained model expects.

In [ ]:
df_hist.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSpend
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [ ]:
"""Define Snapshot Date (CRITICAL)
All recency-based features depend on this.
"""
snapshot_date = INFERENCE_CUTOFF_DATE

In [ ]:
"""RECENCY Feature

Days since last purchase before cut-off.

smaller value → more recent → higher CLV tendency
"""
recency = (
    df_hist
    .groupby("CustomerID")["InvoiceDate"]
    .max()
    .apply(lambda x: (snapshot_date - x).days)
)

In [ ]:
"""FREQUENCY Feature

Number of unique invoices (transactions).

Higher frequency → loyal customer
"""

frequency = (
    df_hist
    .groupby("CustomerID")["InvoiceNo"]
    .nunique()
)


In [ ]:
"""MONETARY Features
Total historical spend
"""

monetary = (
    df_hist
    .groupby("CustomerID")["TotalSpend"]
    .sum()
)


In [ ]:
"""Average order value (AOV)"""
avg_order_value = (
    df_hist
    .groupby("CustomerID")["TotalSpend"]
    .mean()
)


In [ ]:
"""BASKET / PRODUCT FEATURES
These capture purchase diversity & stability."""

#Unique products purchased
unique_products = (
    df_hist
    .groupby("CustomerID")["StockCode"]
    .nunique()
)


#Average quantity per order
avg_quantity = (
    df_hist
    .groupby("CustomerID")["Quantity"]
    .mean()
)


#Variability in order value
order_value_std = (
    df_hist
    .groupby("CustomerID")["TotalSpend"]
    .std()
)

In [ ]:
#Combine All Features (Single Customer Table)
customer_features = pd.DataFrame({
    "recency": recency,
    "frequency": frequency,
    "monetary": monetary,
    "avg_order_value": avg_order_value,
    "unique_products": unique_products,
    "avg_quantity": avg_quantity,
    "order_value_std": order_value_std
}).reset_index()


In [ ]:
#Handle Missing Values

#Some customers may have:
#Only one order → std = NaN

customer_features.fillna(0, inplace=True)

In [ ]:
customer_features.head()

,CustomerID,recency,frequency,monetary,avg_order_value,unique_products,avg_quantity,order_value_std
0,12346,316,12,77556.46,2281.072353,27,2184.852941,13234.943418
1,12347,30,7,5408.50,22.349174,123,12.785124,20.919124
2,12348,66,5,2019.40,39.596078,25,53.215686,44.205549
3,12349,9,4,4428.69,25.306800,138,9.280000,33.761403
4,12350,301,1,334.40,19.670588,17,11.588235,7.275538


In [ ]:
print("Rows:", customer_features.shape[0])
print("Unique customers:", customer_features["CustomerID"].nunique())


Rows: 5850
Unique customers: 5850


In [ ]:
"""

“For inference, I aggregate transaction-level data into customer-level RFM and basket features using only historical data prior to the cut-off date.
These features capture recency, loyalty, spending power, and purchasing diversity.”"""

'\n\n“For inference, I aggregate transaction-level data into customer-level RFM and basket features using only historical data prior to the cut-off date.\nThese features capture recency, loyalty, spending power, and purchasing diversity.”'

# FEATURE TRANSFORMATION & SCALING (INFERENCE)

In [ ]:
FEATURE_COLUMNS = [
    "recency",
    "frequency",
    "monetary",
    "avg_order_value",
    "unique_products",
    "avg_quantity",
    "order_value_std"
]

In [ ]:
customer_features[FEATURE_COLUMNS].head()

,recency,frequency,monetary,avg_order_value,unique_products,avg_quantity,order_value_std
0,316,12,77556.46,2281.072353,27,2184.852941,13234.943418
1,30,7,5408.50,22.349174,123,12.785124,20.919124
2,66,5,2019.40,39.596078,25,53.215686,44.205549
3,9,4,4428.69,25.306800,138,9.280000,33.761403
4,301,1,334.40,19.670588,17,11.588235,7.275538


In [ ]:
import numpy as np

customer_features["frequency"] = np.log1p(customer_features["frequency"])
customer_features["monetary"] = np.log1p(customer_features["monetary"])
customer_features["avg_order_value"] = np.log1p(customer_features["avg_order_value"])
customer_features["order_value_std"] = np.log1p(customer_features["order_value_std"])

#Apply only transforms used during training

In [ ]:
#Separate CustomerID from Features
X = customer_features[FEATURE_COLUMNS]
customer_ids = customer_features["CustomerID"]

In [ ]:
import joblib
pipeline = joblib.load("/content/best_clv_model.pkl")

In [ ]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['recency', 'frequency',
                                                   'total_quantity',
                                                   'total_spend',
                                                   'avg_order_value',
                                                   'unique_products']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['country'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=No...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=400, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [ ]:
pipeline.named_steps

{'preprocessor': ColumnTransformer(transformers=[('num', StandardScaler(),
                                  ['recency', 'frequency', 'total_quantity',
                                   'total_spend', 'avg_order_value',
                                   'unique_products']),
                                 ('cat', OneHotEncoder(handle_unknown='ignore'),
                                  ['country'])]),
 'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None,

In [ ]:
preprocessor = pipeline.named_steps["preprocessor"]
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['recency', 'frequency', 'total_quantity',
                                  'total_spend', 'avg_order_value',
                                  'unique_products']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['country'])])

In [ ]:
preprocessor.transformers_

[('num',
  StandardScaler(),
  ['recency',
   'frequency',
   'total_quantity',
   'total_spend',
   'avg_order_value',
   'unique_products']),
 ('cat', OneHotEncoder(handle_unknown='ignore'), ['country'])]

In [ ]:
frequency = (
    df_hist
    .groupby("CustomerID")["InvoiceNo"]
    .nunique()
)


In [ ]:
total_spend = (
    df_hist
    .groupby("CustomerID")["TotalSpend"]
    .sum()
)


In [ ]:
total_quantity = (
    df_hist
    .groupby("CustomerID")["Quantity"]
    .sum()
)


In [ ]:
unique_products = (
    df_hist
    .groupby("CustomerID")["StockCode"]
    .nunique()
)


In [ ]:
country = (
    df_hist
    .groupby("CustomerID")["Country"]
    .agg(lambda x: x.mode()[0])
)


In [ ]:
customer_features = pd.DataFrame({
    "CustomerID": recency.index,
    "recency": recency.values,
    "frequency": frequency.values,
    "total_quantity": total_quantity.values,
    "total_spend": total_spend.values,
    "avg_order_value": avg_order_value.values,
    "unique_products": unique_products.values,
    "country": country.values
})


In [ ]:
customer_features.fillna(0, inplace=True)


In [ ]:
X = customer_features.drop(columns=["CustomerID"])


In [ ]:
X.columns.tolist()


['recency',
 'frequency',
 'total_quantity',
 'total_spend',
 'avg_order_value',
 'unique_products',
 'country']

In [ ]:
pipeline = joblib.load("/content/best_clv_model.pkl")

clv_predictions = pipeline.predict(X)

In [ ]:
final_predictions = pd.DataFrame({
    "CustomerID": customer_features["CustomerID"],
    "Predicted_30Day_CLV": clv_predictions
})

final_predictions.head()


,CustomerID,Predicted_30Day_CLV
0,12346,27.292959
1,12347,165.090332
2,12348,-7.435968
3,12349,262.171539
4,12350,21.447035


# MODEL INFERENCE & OUTPUT GENERATION

In [ ]:
import joblib
model = joblib.load("/content/best_clv_model.pkl")

In [ ]:
clv_predictions = model.predict(X)

In [ ]:
clv_predictions.shape

(5850,)

In [ ]:
#Build Final Inference Output Table
final_predictions = pd.DataFrame({
    "CustomerID": customer_ids.values,
    "Predicted_30Day_CLV": clv_predictions
})


In [ ]:
#Sanity Checks (VERY IMPORTANT)
#1️⃣ No negative CLV
(final_predictions["Predicted_30Day_CLV"] < 0).sum()

np.int64(341)

In [ ]:
#Business intuition check
final_predictions.sort_values(
    "Predicted_30Day_CLV",
    ascending=False
).head(10)


,CustomerID,Predicted_30Day_CLV
5664,18102,23605.029297
2670,15061,22686.625000
4272,16684,20159.054688
67,12415,18737.605469
2523,14911,17599.662109
2263,14646,15900.536133
5513,17949,14091.141602
5085,17511,13766.512695
3626,16029,12996.908203
4016,16422,9003.891602


In [ ]:
#Save Predictions (Batch Inference)
final_predictions.to_csv(
    "clv_inference_predictions.csv",
    index=False
)


'This file is now:\nReady for marketing teams\nReady for dashboards\nReady for CRM ingestion\n'

In [ ]:
final_predictions["CLV_Segment"] = pd.qcut(
    final_predictions["Predicted_30Day_CLV"],
    q=4,
    labels=["Low", "Medium", "High", "Very High"]
)
